In [1]:
!python -m pip install torch torchvision --index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://download.pytorch.org/whl/cu126
INFO: pip is looking at multiple versions of networkx to determine which version is compatible with other requirements. This could take a while.
  Using cached https://download.pytorch.org/whl/networkx-3.3-py3-none-any.whl.metadata (5.1 kB)
  Using cached https://download.pytorch.org/whl/mpmath-1.3.0-py3-none-any.whl (536 kB)
   ---------------------------------------- 0.0/2.6 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.6 GB 5.6 MB/s eta 0:07:43
   ---------------------------------------- 0.0/2.6 GB 5.9 MB/s eta 0:07:17
   ---------------------------------------- 0.0/2.6 GB 5.6 MB/s eta 0:07:42
   ---------------------------------------- 0.0/2.6 GB 5.9 MB/s eta 0:07:20
   ---------------------------------------- 0.0/2.6 GB 5.6 MB/s eta 0:07:42
   ---------------------------------------- 0.0/2.6 GB 6.0 MB/s eta 0:07:12
   ---------------------------------------- 0.0/2.6 GB 6.0 MB/s eta 0:07:10
   ---------

In [1]:
import zipfile
from PIL import Image
import torch
from torchvision import transforms
from io import BytesIO
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from collections import defaultdict



device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("GPU is", "available" if device else "NOT AVAILABLE")

# Caminho para o arquivo ZIP
zip_path1 = 'cops_db.zip'

# Transforms para redimensionar e converter para tensor
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),  # Converte para [C x H x W]
])

# Lista para armazenar os tensores
image_in_tensors = []
image_out_tensors = []
image_out_class = []


# Abre o ZIP e processa as imagens diretamente
def loadImages_crops(zip_path, max):
    count = 0
    label_map = {}  # Mapeia nome da pasta para um label numérico
    image_in_tensors = []
    image_out_class = []

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for file_name in zip_ref.namelist():
            if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                # Extrai o nome da pasta (classe)
                parts = file_name.split('/')
                if len(parts) < 2:
                    continue  # Ignora arquivos fora das pastas de classe
                folder_name = parts[1]
                #print(folder_name)    

                # Atribui um label único para cada pasta
                if folder_name not in label_map:
                    label_map[folder_name] = len(label_map)
                label = label_map[folder_name]

                #print(label)                 

                with zip_ref.open(file_name) as file:
                    image = Image.open(BytesIO(file.read())).convert('RGB')
                    tensor = transform(image)  # Shape: [3, 256, 256]
                    image_in_tensors.append(tensor)
                    image_out_class.append([label])

                count += 1
                if count % 1000 == 0:
                    print(count)

                if count >= max:
                    break
    
    print("NClasses: ",len(label_map))
    return image_in_tensors, image_out_class


# Empilha os tensores em um batch
image_in_tensors,image_out_class = loadImages_crops(zip_path1,5000)


print(f'Total de imagens carregadas: {len(image_in_tensors)}')

class_groups = defaultdict(list)
for img, lbl in zip(image_in_tensors, image_out_class):
    class_groups[int(lbl[0])].append(img)

train_images = []
train_labels = []
test_images = []
test_labels = []

for label, images in class_groups.items():
    images = np.array(images)
    n = len(images)
    indices = np.random.permutation(n)
    split = int(n * 0.7)

    train_idx = indices[:split]
    test_idx = indices[split:]

    for i in train_idx:
        train_images.append(images[i])
        classes_obj = np.zeros(30)
        classes_obj[label] = 1
        train_labels.append(classes_obj)

    for i in test_idx:
        test_images.append(images[i])
        classes_obj = np.zeros(30)
        classes_obj[label] = 1
        test_labels.append(classes_obj)

# Empacotar como tensores
train_images = [torch.tensor(img) if not isinstance(img, torch.Tensor) else img for img in train_images]

t_xt = torch.stack(train_images).to(device)
t_yt = torch.tensor(train_labels, dtype=torch.float32).to(device)

test_images = [torch.tensor(img) if not isinstance(img, torch.Tensor) else img for img in test_images]
t_xv = torch.stack(test_images).to(device)
t_yv = torch.tensor(test_labels, dtype=torch.float32).to(device)


dataset = TensorDataset(t_xt, t_yt)
#batch_tensor = torch.stack(image_in_tensors)  # Shape: [N, 3, 256, 256]


print(f'Shape do batch: {t_xt.shape}')
print(f'Shape dos rótulos: {t_yt.shape}')



GPU is available
NClasses:  30
Total de imagens carregadas: 829
Shape do batch: torch.Size([569, 3, 128, 128])
Shape dos rótulos: torch.Size([569, 30])


C:\Users\Denni\AppData\Local\Temp\ipykernel_15284\2423188447.py:111: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:256.)
  t_yt = torch.tensor(train_labels, dtype=torch.float32).to(device)


In [2]:
import torch;
import torch.nn as nn

class RedeCnnBirdNotBird(nn.Module):
    def __init__(self):
        super(RedeCnnBirdNotBird, self).__init__()

        self.conv1 = nn.Conv2d(3, 6, 5, stride=1) # 124x124
        self.conv2 = nn.Conv2d(6, 12, 5, stride=1) # 120x120
        self.poll1 = nn.MaxPool2d(2,2) # 60 x 60
        self.conv3 = nn.Conv2d(12, 24, 5, stride=1) # 56 x 56
        self.conv4 = nn.Conv2d(24, 48, 5, stride=1) # 52 x 52
        self.poll2 = nn.MaxPool2d(2,2) # 26x26
        self.conv5 = nn.Conv2d(48, 96, 11, stride=1) # 16 x 16
        self.poll3 = nn.MaxPool2d(2,2) # 8x8
        
        self.linear1 = nn.Linear(6144,512)
        self.linear2 = nn.Linear(512,30)
        #self.linear3 = nn.Linear(164,164)
        #self.linear4 = nn.Linear(512,1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.poll1(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.poll2(x)
        x = self.conv5(x)
        x = self.poll3(x)
        #print("x poll1: ",x.size())
        
        x = torch.flatten(x, start_dim=1)
        
        #print("x flatten: ",x.size())
        
        x = torch.relu(self.linear1(x))
        x = torch.relu(self.linear2(x))
        
        #x = torch.relu(self.linear1(x))
        #x = torch.relu(self.linear2(x))
        #x = torch.tanh(self.linear3(x))
        #x = torch.relu(self.linear4(x))
        return x

In [11]:
import time

def train(cnn,dataset, epochs=10):
    opt = torch.optim.Adam(cnn.parameters(),lr=0.00001)#0.00001  #0.0000001

    train_loader = torch.utils.data.DataLoader(
        dataset, batch_size=8, shuffle=True
    )

    for epoch in range(epochs):
        totalloss = 0
        batch = 0
        start_time = time.time()
        opt.zero_grad()

        for inputs, targets in train_loader:
            
            #inputs = x[:, :insize]    
            #targets = x[:, insize:] 
            #x = x.to(device) # GPU
            x_hat = cnn(inputs)
            #print(inputs.size()," ",targets.size()," ",x_hat.size())
            loss = ((targets - x_hat)**2).sum()

            loss.backward(retain_graph=True)
            totalloss+=loss
            batch+=1
 
            opt.step()
            opt.zero_grad()
            #print("step: ")

        end_time = time.time()
        print(epoch," Total Loss: ",(totalloss/len(dataset))," time ",(end_time-start_time))
        #if epoch%250==0:
        #    torch.save(cnn, "backup_training_gam_v02.pth")
    return cnn


In [13]:
import os

cnn = RedeCnnBirdNotBird().to(device)

if os.path.exists("cnn_v1.pth"):
   cnn = torch.load("cnn_v1.pth", weights_only=False)
   print("CARREGOU MODELO: ","cnn_v1.pth")

cnn = train(cnn, dataset,epochs=100)

torch.save(cnn, "cnn_v1.pth")





0  Total Loss:  tensor(0.9856, device='cuda:0', grad_fn=<DivBackward0>)  time  0.9781875610351562
1  Total Loss:  tensor(0.9781, device='cuda:0', grad_fn=<DivBackward0>)  time  0.7311253547668457
2  Total Loss:  tensor(0.9729, device='cuda:0', grad_fn=<DivBackward0>)  time  0.7353694438934326
3  Total Loss:  tensor(0.9640, device='cuda:0', grad_fn=<DivBackward0>)  time  0.7283148765563965
4  Total Loss:  tensor(0.9561, device='cuda:0', grad_fn=<DivBackward0>)  time  0.7319431304931641
5  Total Loss:  tensor(0.9466, device='cuda:0', grad_fn=<DivBackward0>)  time  0.735485315322876
6  Total Loss:  tensor(0.9365, device='cuda:0', grad_fn=<DivBackward0>)  time  0.7379105091094971
7  Total Loss:  tensor(0.9273, device='cuda:0', grad_fn=<DivBackward0>)  time  0.7392868995666504
8  Total Loss:  tensor(0.9143, device='cuda:0', grad_fn=<DivBackward0>)  time  0.7372937202453613
9  Total Loss:  tensor(0.9043, device='cuda:0', grad_fn=<DivBackward0>)  time  0.7345607280731201
10  Total Loss:  tens

In [8]:
def calculaMetricas(dataset,cnn):
    tabela_verdade = [
        [0, 0],  # classe 0
        [0, 0]   # classe 1
    ]

    train_loader = torch.utils.data.DataLoader(
        dataset, batch_size=1, shuffle=True
    )

    acertos = 0
    erros = 0
    for inputs, targets in train_loader:
        x_hat = cnn(inputs)
        
        classe_predita = torch.argmax(x_hat[0]).item()

        if  targets[0][classe_predita] == 1:
            acertos+=1
        else:
            erros+=1 
        #print(targets[0][0])
        #tabela_verdade[classe][int(targets[0][0])]+=1
     
    print("Acertos: ",acertos)
    print("Erros: ",erros) 
    print(tabela_verdade)

    print('   B','   NB')
    print('PB ',tabela_verdade[1][1],' ',tabela_verdade[1][0])
    print('PNB ',tabela_verdade[0][1],' ',tabela_verdade[0][0])

    VP = tabela_verdade[1][1]
    FP = tabela_verdade[1][0]
    FN = tabela_verdade[0][1]
    VN = tabela_verdade[0][0]
    Total = VP+FP+FN+VN
    ACC = (VP+VN)/Total
    PRE = VP/(VP+FP)
    REV = VP/(VP+FN)
    F1 = (PRE*REV/(PRE+REV))*2

    print("ACC ",ACC)
    print("PRE ",PRE)
    print("REV ",REV)
    print("F1 ",F1)

In [9]:
print("------------------- Treinamento ---------------------------------")
calculaMetricas(dataset,cnn)
print("------------------- TESTE ---------------------------------")
t_xtb = xb.to(device)
t_ytb = yb.to(device)
datasetb = TensorDataset(t_xtb, t_ytb)
calculaMetricas(datasetb,cnn)

------------------- Treinamento ---------------------------------
Acertos:  345
Erros:  224
[[0, 0], [0, 0]]
   B    NB
PB  0   0
PNB  0   0


ZeroDivisionError: division by zero